In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import sys
import os
sys.path.append(os.path.abspath("../../.."))  # Adds the project root to sys.path
import numpy as np 
import math
from scipy.sparse import coo_matrix
from scipy.sparse import csr_matrix
import implicit
from sklearn.model_selection import train_test_split

from recommenders.datasets.shared import *

In [12]:
save_interaction_values("../../../../pa-data/Interactions.xlsx")

In [2]:
Y, R, num_products = get_data()
sparse_product_user = csr_matrix(Y)
# print(sparse_product_user)
product_names = load_csv_list('Products.csv')
product_names = np.array([row[0] for row in product_names])
# print(product_names)
user_ids = load_csv_list('Users.csv')[0]
# print(user_ids)

# get the transpose since the most of the functions in implicit expect (user, product) sparse matrices instead of (product, user)
sparse_user_product = sparse_product_user.T.tocsr()
print(sparse_user_product)


Y (117, 4) R (117, 4)
num_products 117
num_users 4
  (0, 0)	0.22213805503464382
  (0, 1)	0.3313910367866774
  (0, 2)	0.15653804165619362
  (0, 4)	0.15653804165619362
  (0, 5)	0.15653804165619362
  (0, 6)	0.22213805503464382
  (0, 7)	0.22213805503464382
  (0, 9)	0.15653804165619362
  (0, 10)	0.15653804165619362
  (0, 11)	0.5271427432599853
  (0, 13)	0.1645639186764624
  (0, 16)	0.15653804165619362
  (0, 17)	0.47026626619864825
  (0, 18)	0.15653804165619362
  (0, 23)	0.4943773332553122
  (0, 24)	0.4943773332553122
  (0, 25)	0.22213805503464382
  (0, 32)	0.21130425425031218
  (0, 35)	0.1645639186764624
  (0, 36)	0.21130425425031218
  (0, 39)	0.34838181857191763
  (0, 40)	0.34838181857191763
  (0, 42)	0.23352731666310758
  (0, 45)	0.1730012912109395
  (0, 48)	0.1730012912109395
  :	:
  (3, 1)	0.22213805503464382
  (3, 26)	0.1645639186764624
  (3, 28)	0.23352731666310758
  (3, 37)	0.23352731666310758
  (3, 53)	0.1730012912109395
  (3, 63)	0.12816250873052792
  (3, 64)	0.12816250873052792
  

In [3]:
model = implicit.als.AlternatingLeastSquares(factors=100, regularization=0.1, alpha=1.0)
model.fit(sparse_user_product)

c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\implicit\cpu\als.py:95: RuntimeWarning: OpenBLAS is configured to use 12 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/15 [00:00<?, ?it/s]

In [10]:
flat_user_ids = user_ids
user_id = (str)(55284)
user_index = flat_user_ids.index(user_id)
indexes, scores = model.recommend(user_index, sparse_user_product[user_index], N=10, filter_already_liked_items=False)

In [12]:
import numpy as np
import pandas as pd

original_scores = Y[:, user_index]

pd.DataFrame({
    "product": product_names[indexes], 
    "score": scores, 
    "original_score": original_scores[indexes],
    "already_liked": np.in1d(indexes, sparse_user_product[user_index].indices)
})

,product,score,original_score,already_liked
0,Vill 074376,0.966265,0.683233,True
1,EN 4512042,0.948619,0.442119,True
2,RUBI 01958,0.946707,0.425515,True
3,HT1A770,0.946704,0.425515,True
4,STHT0-05926,0.922423,0.285231,True
5,MAX TW1061,0.922418,0.285231,True
6,SENCO AX10EAAP,0.922412,0.285231,True
7,014T,0.917727,0.222138,True
8,71 02 200,0.906873,0.233527,True
9,6-PHT150,0.906869,0.233527,True


In [6]:
indexes, scores = model.similar_items(100)
pd.DataFrame({"product": product_names[indexes], "score": scores})

,product,score
0,VILL 055320,1.000000
1,VILL 058189,1.000000
2,UN120/1MS 6-22/17,1.000000
3,WBS 1214526,0.999999
4,VILL 016432,0.999999
5,EN 3410810,0.999999
6,3E 1130864,0.999999
7,RUBI 01958,0.999999
8,EN 4512042,0.999999
9,W 26728,0.999999
